# 정규표현식(Regular Expressions)

---
## 메타문자
 - +, *, [ ], { } 등의 메타문자 : 매치가 진행될 때 현재 매치되고 있는 문자열의 위치가 변경(문자열 소비)
 - |, ^, $, \A, \Z, \b, \B : 문자열을 소비x

==================================================
### | : or과 동일한 의미

In [3]:
import re
p = re.compile('Crow|Servo')
m = p.match('CrowHello')
print(m)

<re.Match object; span=(0, 4), match='Crow'>


==================================================
### ^
- 문자열의 맨 처음과 일치함을 의미
- re.MULTILINE을 사용할 경우에는 여러 줄의 문자열일 때 각 줄의 처음과 일치함을 의미

In [4]:
print(re.search('^Life', 'Life is too short'))

<re.Match object; span=(0, 4), match='Life'>


In [5]:
print(re.search('^Life', 'My Life'))

None


==================================================
### $
 - 문자열의 끝과 매치함을 의미

In [6]:
print(re.search('short$', 'Life is too short'))

<re.Match object; span=(12, 17), match='short'>


In [7]:
print(re.search('short$', 'Life is too short, you need python'))

None


==================================================
### \A, \Z
 - ^, $ 메타 문자와 동일한 의미
 - re.MULTILINE 옵션을 사용할 경우 줄과 상관없이 전체 문자열의 처음, 마지막하고만 매치(^, $와 다른점)

==================================================
### \b
 - 단어 구분자(Word boundary)
 - 보통 단어는 whitespace에 의해 구분
 - \b는 백스페이스(BackSpace)를 의미 -> r'\bclass\b' r을 반드시 붙여서 사용

In [8]:
p = re.compile(r'\bclass\b')
print(p.search('no class at all')) 

<re.Match object; span=(3, 8), match='class'>


In [9]:
print(p.search('the declassified algorithm'))

None


==================================================
### \B
 - \b 메타 문자와 반대의 경우
 - whitespace로 구분된 단어가 아닌 경우에만 매치

In [10]:
p = re.compile(r'\Bclass\B')
print(p.search('no class at all')) 

None


In [11]:
print(p.search('the declassified algorithm'))

<re.Match object; span=(6, 11), match='class'>


In [12]:
print(p.search('one subclass is'))

None


---
## 그루핑(Grouping) : ( )
|group(인덱스)|설명|
|:--:|--|
|group(0)|매치된 전체 문자열|
|group(n)|n 번째 그룹에 해당되는 문자열|


In [13]:
p = re.compile('(ABC)+')
m = p.search('ABCABCABC OK?')
print(m)

<re.Match object; span=(0, 9), match='ABCABCABC'>


In [14]:
print(m.group())

ABCABCABC


In [15]:
p = re.compile(r"\w+\s+\d+[-]\d+[-]\d+")
            # 단어,공백,숫자,'-',숫자,'-',숫자
m= p.search("park 010-1234-1234")
m

<re.Match object; span=(0, 18), match='park 010-1234-1234'>

### 그루핑 
 - ( ) 사용 
 - 그룹을 중첩되게 사용하는 것도 가능
 - 바깥쪽부터 시작하여 안쪽으로 들어갈수록 인덱스가 증가

In [16]:
p = re.compile(r"(\w+)\s+((\d+)[-]\d+[-]\d+)")
m = p.search("park 010-1234-1234")
m.group(1) , m.group(2), m.group(3)

('park', '010-1234-1234', '010')

==================================================
### 그루핑된 문자열 재참조

In [17]:
# 재참조 메타문자 \n : 정규식의 그룹 중 n번째 그룹을 의미
p = re.compile(r'(\b\w+)\s+\1')
            # r'(\b\w+)\s+(\b\w+)'과 같은 의미
p.search('Paris in the the spring').group()

'the the'

==================================================
### 그루핑된 문자열에 이름 붙이기
 - 형식 : (?P<그룹명>...)
 - (\w+)에 이름 -> (?P<name>\w+)

In [18]:
p = re.compile(r"(?P<name>\w+)\s+((\d+)[-]\d+[-]\d+)")
m = p.search("park 010-1234-1234")
print(m.group("name"))

park


In [19]:
# 재참조 가능 (?P=그룹이름)
p = re.compile(r'(?P<word>\b\w+)\s+(?P=word)')
p.search('Paris in the the spring').group()

'the the'

---
## 전방(Lookahead)탐색과 후방(Lookbehind)탐색
 - 조건에 해당하는 패턴의 전,후방을 탐색
 - 조건이 통과되어도 문자열이 소비되지 않음(조건패턴은 반환x)  
 
|종류|설명|
|:--:|:--:|
|(?=)|긍정형 전방탐색|
|(?!)|부정형 전방탐색|
|(?<=)|부정형 후방탐색|
|(?<!)|부정형 후방탐색|

==================================================
### 긍정형 전방탐색 
 - (?=...) : ...에 해당하는 정규식과 매치되면 앞(전방)을 탐색

In [20]:
# 일반 : ':'까지 소비
p = re.compile(".+:")
m = p.search("http://google.com")
m.group()

'http:'

In [21]:
# 긍정형 전방탐색 : ':'은 소비x
p = re.compile(".+(?=:)")
m = p.search("http://google.com")
m.group()

'http'

==================================================
### 부정형 전방탐색 
 - (?!...) : ...에 해당하는 정규식과 매치되지 않으면 뒤(후방)을 탐색
 - 예시)
 
```.*[.](?!bat$).*$ ```  -> .bat로 끝나지 않으면 전방 탐색
 
```.*[.](?!bat$|exe$).*$``` -> .bat, .exe.로 끝나지 않으면 전방 탐색

==================================================
### 긍정형 후방탐색 
 - (?<=...) : ...에 해당하는 정규식과 매치되면 뒤(후방)를 탐색
 - 일반 :  '\$[0-9.]+' -> '$숫자' 반환
 - 긍정형 후방탐색  
 '(?<=\$)[0-9.]+' -> '$숫자'를 만나면 $뒤의 '숫자'만 반환
 

==================================================
### 부정형 후방탐색 
 - (?<=!...) : ...에 해당하는 정규식과 매치되지 않으면 뒤(후방)를 탐색

---
## 문자열 바꾸기 : sub메서드
 - 정규표현식조건.sub(대체할문자열,대상문자열,[count=n])
 - 대상문자열 중 조건에 맞는 문자열을 대체할문자열로 n번 바꾸기
 - count 미입력시 전부 대체

In [22]:
# 'blue', 'white', 'red' 이면 'colour'로 바꾸기
p = re.compile('(blue|white|red)')
p.sub('colour', 'blue socks and red shoes')

'colour socks and colour shoes'

In [23]:
# 'blue', 'white', 'red' 이면 'colour'로 바꾸기 count=1
p = re.compile('(blue|white|red)')
p.sub('colour', 'blue socks and red shoes',count=1)

'colour socks and red shoes'

==================================================
### subn 메서드
 - 반환 결과 튜플로 돌려줌 (변경된 문자열, 바꾸기 발생 횟수)

In [25]:
p = re.compile('(blue|white|red)')
p.subn( 'colour', 'blue socks and red shoes')

('colour socks and colour shoes', 2)

==================================================
### sub 메서드 사용 시 참조 구문 사용하기

In [26]:
# 이름+전화번호 -> 전화번호+이름
p = re.compile(r"(?P<name>\w+)\s+(?P<phone>(\d+)[-]\d+[-]\d+)")

# 그룹이름 참조 : sub의 바꿀 문자열 부분에 \g<그룹이름>을 사용
p.sub("\g<phone> \g<name>", "park 010-1234-1234")

'010-1234-1234 park'

In [27]:
p = re.compile(r"(?P<name>\w+)\s+(?P<phone>(\d+)[-]\d+[-]\d+)")

# 번호를 사용해 참조
print(p.sub("\g<2> \g<1>", "park 010-1234-1234"))


010-1234-1234 park


==================================================
### sub 메서드의 매개변수로 함수 넣기
 - sub의 첫 번째 매개변수로 함수를 사용할 경우  
 -> 해당 함수의 첫 번째 매개변수에는 정규식과 매치된 match 객체가 입력
 - 매치되는 문자열은 함수의 반환 값으로 바뀜

In [36]:
# 객체를 입력받아 16진수로 반환하는 함수(match객체 입력)
def hexrepl(match):
    value = int(match.group())
    return hex(value)

# 숫자만 매치
p = re.compile(r'\d+')

# 65490, 49152만 매치 후 함수에 적용
p.sub(hexrepl, 'Call 65490 for printing, 49152 for user code.')

'Call 0xffd2 for printing, 0xc000 for user code. 0xd9ed'

---
## Greedy vs Non-Greedy
 - ?, *?, +?, ??, {m,n}? 와 같이 사용
 - 최소한의 반복을 수행하도록 도와줌

In [37]:
# <html>만 추출의도
s = '<html><head><title>Title</title>'
len(s)

32

In [38]:
re.match('<.*>', s).span()

(0, 32)

In [39]:
# Greedy
# 매치할 수 있는 최대한의 문자열인 <html><head><title>Title</title> 문자열을 모두 소비
re.match('<.*>', s).group()

<html><head><title>Title</title>


In [40]:
# nom_greedy 문자 ? -> *의 탐욕을 제한
re.match('<.*?>', s).group()

'<html>'